# Agente Auto


This notebook presents an agent-based model that simulates the propagation of a disease through a network.
It demonstrates how to use the [agentpy](https://agentpy.readthedocs.io) package to create and visualize networks, use the interactive module, and perform different types of sensitivity analysis. 

In [61]:
# Model design
import agentpy as ap
import networkx as nx 
import random 

# Visualization
import matplotlib.pyplot as plt 
import seaborn as sns
import IPython

## About the model



## Defining the model

In [62]:
class Car(ap.Agent):
    
    def setup(self):  
        """ Initialize a new variable at agent creation. """
        self.condition = 0  # Conduciendo = 0, Standby = 1
        self.dir = 0 #Horizontal = 0, Vertical = 1
        self.grid = self.model.grid
        
        
    def drive(self):
        """ Manejar """
        if self.condition == 0 and self.dir == 0:  #Horizontal
            self.grid.move_by(self, (0,1))
        if self.condition == 0 and self.dir == 1: #Vertical
            self.grid.move_by(self, (1,0))

    def turn(self):
        "Girar"
        self.dir = 1

        

In [63]:
class CarModel(ap.Model):
    
    def setup(self):
        """ Initialize the agents and network of the model. """
        self.grid = ap.Grid(self, (self.p.gridsize, self.p.gridsize), torus= True, track_empty= True)
        self.agents = ap.AgentList(self, self.p.cars, Car)
        self.grid.add_agents(self.agents, random=True, empty=True)
        
    def step(self):   
        """ Define the models' events per simulation step. """
        self.agents.drive()

    
        

## Running a simulation

In [64]:
parameters = {  
    'gridsize': 25,
    'cars': 4,
    'steps': 100,
    
}

model = CarModel(parameters)
results = model.run() 

Completed: 100 steps
Run time: 0:00:00.019006
Simulation finished


## Analyzing results

In [65]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 3 keys
'reporters': DataFrame with 1 variable and 1 row
}

## Creating an animation

In [66]:
def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('condition')
    ap.gridplot(attr_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Car model \n Time-step: {model.t}")

fig, ax = plt.subplots()
model = CarModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())